# Pràctica 1: Resolem un problema de classificacio

## Objectius

Els objectius d'aquesta pràctica són:

* Aplicar els coneixements adquirits sobre classificacio, tecniques d'emplenats de nans i validacio creuada.
  
* Ser capac de comparar diferents models classificacio

* Ser capac de fer busca d'hiperparametres.

* Entendre i implamentar la validacio creuada

* Analitzar detalladament els resultats obtinguts durant l'aprenentatge dels diferents models.

Aquesta practica es previa al cas kaggle que realitzareu durant la segona part de l'assignatura. En aquesta primera practica les preguntes estan definides, pero us ha de servir d'aprenentatge alhora de saber com estructurar un projecte d'aprenentatge automatic ja que en el cas kaggle no tindreu les preguntes.

## Bases de dades

En aquesta practica farem servir la base de dades del titanic. L'atribut que predirem es Survived, el qual ens diu si va sobreviure o no cada passatger.


https://www.kaggle.com/c/titanic/data


##Treball en grup
Aquesta practica es treballara en grups de 2-3 persones. En casos excepcionals i degudament justificats la practica es podra realitzar de forma individual.
## Seguiment i entrega de la pràctica

En la pràctica 1, es presenten diverses tasques per fer una correcta comparativa dels resultats obtinguts per diversos mètodes de classificació numèrica en una mateixa base de dades.


En aquesta practica es realitzara sessions de seguiment del treball. Aquestes sessions de treball està orientada a que els alumnes que vingueu pugueu preguntar i resoldre dubtes sobre les dades, preguntar sobre l'objectiu de cada apartat dels enunciats que no us hagi quedat clar, i preguntar sobre els resultats que esteu obtenint a l'hora d'analitzar les dades. Es molt recomanable venir amb el treball fet a clase per tal de poder comentar dubtes.

Pel que fa l'entrega, caldra entregar per caronte el seguent:

1. Memòria en format PDF o en la mateixa notebook explicant els resultats trobats sobre la bases de dades amb el respectiu codi de python. Aquesta nota es la mateixa per tots els membres del grup (9 pts)

2. Presentació amb els resultats 4 min màxim. Aquesta nota es individual. (1 pt)

Nota: En cas d'entregar la memoria com a PDF, caldra entregar el codi de python a part. Si s'entrega com a notebook, la mateixa notebook servira com a codi.



# Descripcio de la pràctica

A continuacio es mostren tots els continguts que s'evaluara en la pràctica:

1. EDA (exploratory data analysis) (1 punts):
  * Analisi de tamany i tipologia de dades ( 0.25 pt)
  * Primera valoracio de correlacions ( 0.5 pt)
  * Analisi atribut target ( 0.25 pt)
2. Preprocessing (2 punts):
  * Eliminacio de nans (0.5 punts)
  * Encoding de categoriques (0.75 punts)
  * Altres (PCA, normalitzacio..)
3. Metric selection (1.5 punts):
  * Seleccio de la millor metrica pel problema (0.75 punts)
  * Visualitzacio de ROC/AUC (0.75 punts)
4. Model Selection amb Crossvalidation (4 punts):
  * Seleccio del millor model (2 punts)
  * Busqueda hiperparametres (2 punts)
5. Analisi final (1.5 punt)

Cal dir que, les puntuacions dins de cada apartat son orientatives. La pràctica esta construida a partir d'un seguit de preguntes orientatives en cada apartat les quals tenen relacio amb els continguts evaluables. **NO cal contestar-les totes**. Són una guia per a que reflexioneu i aprengueu detalls de cada apartat.  Es recomanable, aixo si, llegir totes les preguntes abans de realitzar la practica i tenir-les en ment alhora d'executar-la.


**IMPORTANT**: El que es valorara en la practica es la capacitat de mantenir una narrativa coherent alhora que es realitzen els resultats. No es mirara tant que alguna pregunta quedi per respondre sino que els passos seguits en base als resultats obtinguts siguin coherents.

In [17]:
### Imports
import pandas as pd

### 1. EDA (exploratory data analysis) (1 punt)

Abans de res cal sempre veure com es la base de dades asignada?

**Preguntes:**
* Quants atributs té la vostra base de dades?
* Quin tipus d'atributs tens? (Númerics, temporals, categorics, binaris...)
* Com es el target, quantes categories diferents existeixen?
* Tenim nans en les dades?
* Podeu veure alguna correlació entre X i y?
* Estan balancejades les etiquetes (distribució similar entre categories)? Creus que pot afectar a la classificació la seva distribució?


### Anàlisi de dades

In [18]:
# Llegim la base de dades
df = pd.read_csv('train.csv')

# Obtenim el numero d'atributs
num_atr = df.shape[1]

# Veiem quants atributs te el nostre csv
print(f"La nostra base de dades té {num_atr} atributs.")

La nostra base de dades té 12 atributs.


Veiem doncs que la nostra base de dades disposa de 12 atributs, els quals tenen els següents noms:


In [19]:
atributos = df.columns.tolist()

print(f"Els noms dels atributs son: ")
for nombre in atributos:
    print(nombre)

Els noms dels atributs son: 
PassengerId
Survived
Pclass
Name
Sex
Age
SibSp
Parch
Ticket
Fare
Cabin
Embarked


Ara ens interesa saber el tipus de dades que guarden aquests atributs

In [20]:
atr_type = df.dtypes

print(f"Els tipus de dades dels atributs son:\n{atr_type}.")

Els tipus de dades dels atributs son:
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object.


Ara ens interesa saber com es el target i quantes categories diferents existeixen:
Despres de veure la base de dades, podem deduir que el nostre target a analitzar és la columna "Survived", la qual ens indica si el passatger va sobreviure (1) o no (0), veiem també que les diferents categories possibles(2) son 0, no sobreviu, 1 sobreviu.

In [21]:
num_categories_survived = df['Survived'].nunique()

print(f"El numero de categories que te l'atribut 'Survived' és: {num_categories_survived}")

El numero de categories que te l'atribut 'Survived' és: 2


Executant el codi python anterior, veiem que la nostra deducció de possibles valors categorics es correcte, i son 2.

Ara hem de determinar si la nostra base de dades te o no nans, això ho fariem de la següent manera:

In [22]:
df_nans = df.isna().any().any()

if df_nans:
    print(f"La base de dades té nans.")
else:
    print(f"La base de dades no té nans.")

La base de dades té nans.


Ara hem de veure si hi ha alguna corelació entre X i Y a la hora de determinar l'atribut "Survived".

In [23]:
# Replicamos la base de datos eliminando las columnas que no nos interesan.
dataset_corr = df.copy()
useless_columns = []

for f in dataset_corr:
    if f == 'Name' or f == 'Ticket' or f == 'Cabin':
        useless_columns.append(f)

dataset_corr.drop(useless_columns, axis='columns', inplace=True)

Eliminamos las columnas Name , Ticket y Cabin por los motivos que especificamos a continuación:
- Name: La eliminamos puesto que es un indicador del nombre del pasajero y no es relevante.
- Ticket: La eliminamos puesto que es un indicador del ticket del pasajero y no es relevante.
- Cabin: La eliminamos puesto que es un indicador de la cabina de cada pasajero y tampoco es relevante.


In [24]:
# Convertimos los valores de sex a 0 y 1. ( codificación one-hot )
df_encoded = pd.get_dummies(dataset_corr, columns=['Sex'], drop_first=True)

# Convertimos los valores de Embarked.
maping = {'C' : 0 , 'Q': 1, 'S' : 2}
df_encoded['Embarked_numeric'] = df_encoded['Embarked'].map(maping)
df_encoded = df_encoded.drop(columns=['Embarked'])

# TODO: Preguntar mejor opción para la codificación ( encode )
#df_encoded = pd.get_dummies(df_encoded, columns=['Embarked'], drop_first=False)

El codigo de arriba en primer lugar, eliminar la columna 'Sex', creando una llamada 'Sex_male', la cual es 1 si el usuario es hombre y 0 si mujer, de esta manera podremos calcular la correlación con 'Survived'. En segundo lugar, hemos eliminado la columna 'Embarked', creando la columna 'Embarked_numeric', que sustituie los valores C, Q y S, por 0, 1, 2 respectivamente, consiguiendo así poder calcular la correlación.
A continuación se muestra como queda la base de datos.

In [25]:
print(df_encoded)

     PassengerId  Survived  Pclass   Age  SibSp  Parch     Fare  Sex_male  \
0              1         0       3  22.0      1      0   7.2500         1   
1              2         1       1  38.0      1      0  71.2833         0   
2              3         1       3  26.0      0      0   7.9250         0   
3              4         1       1  35.0      1      0  53.1000         0   
4              5         0       3  35.0      0      0   8.0500         1   
..           ...       ...     ...   ...    ...    ...      ...       ...   
886          887         0       2  27.0      0      0  13.0000         1   
887          888         1       1  19.0      0      0  30.0000         0   
888          889         0       3   NaN      1      2  23.4500         0   
889          890         1       1  26.0      0      0  30.0000         1   
890          891         0       3  32.0      0      0   7.7500         1   

     Embarked_numeric  
0                 2.0  
1                 0.0  
2  

Una vez hemos hecho las modificaciones pertinentes a nuesta base de datos, ya podemos caluclar la correlación con la variable 'Survived'.

In [26]:
new_names = df_encoded.columns.to_list()

corr = {}
for name in new_names: 
    if name == 'Survived' or name == 'PassengerId':
        pass
    else:
        corr_value = df_encoded['Survived'].corr(df_encoded[name])
        corr[name] = corr_value
print(corr)

{'Pclass': -0.3384810359610148, 'Age': -0.07722109457217755, 'SibSp': -0.035322498885735576, 'Parch': 0.0816294070834836, 'Fare': 0.2573065223849622, 'Sex_male': -0.5433513806577547, 'Embarked_numeric': -0.16971767716412958}


Una vez tenemos los resultados pasamos a analizarlos:
- Pclass: -0,338
    - Esta correlación es negativa y moderadamente fuerte. Indica que a medida de que Pclass disminuye ( lo que significa que el pasajero está en una clase mas alta ) la probabilidad de sobrevivir aumenta.
- Age: -0,077
    - Esta correlación es negativa y bastante baja. Lo que indica que a medida que la edad del pasajero aumenta, la probabilidad de sobrebirir del pasajero disminuye ligeramente. Pero como lo correlación es bastante baja podemos deducir que este no es un factor determinante.
- SibSp: -0.035
    - Esta correlación también es negativa y bastante baja. Indica que a medida que el numero de hermanos o cónyuges a bordo aumenta, la probabilidad de sobrevivir disminuye ligeramente. Como la correlación es bastante baja podemos deducir que no es un factor determinante.
- Parch: 0,082
    - Esta correlación es positiva y bastante baja. Indica que a medida que el numero padres o hijos a bordo aumenta, la posibilidad de sobrevivir aumenta ligeramnete. Como la correlación es bastante baja podemos deducir que no es un factor determinante.
- Fare: 0.257
    - Esta correlación es positiva y relativamente fuerte. Indica que a medida que el coste del billete aumenta, la posibilidad de sobrevivir aumente moderadamente. Como esta correlación es moderada podemos deducir que puede ser un factor determinante, puesto que los pasageros que havian pagado mas, quizas tenian prioridad a la hora de ser rescatados.
- Sex_male: -0,543
    - Esta correlación es negativa y fuerte. Indica que a ser del sexo masculino, disminuia drasticamente las posibilidades de sobrevivir, por lo que, las mujeres, tenian prioridad a la hora del rescate. Esta correlación al ser bastante fuerte podemos decir que es un factor determinante de la supervivencia del pasagero.
- Embarked_numeric: -0,170
    - Esta correlación es negativa y moderadamente baja. Indica que el puerto de embarque está asociado a la probabilidad de sobrevivir, pero al ser moderadamente baja, no es un factor determinante.


Una vez hemos analizado la correlación, necesitamos saber si las etiquetas están balanceadas y si puede afectar a la clasificación su distribución.


### 2. Preprocessing (normalitzation, outlier removal, feature selection..) (2 punts)
Un cop vistes les dades de les que es disposa, cal preparar les dades per als nostres algoritmes. Segons la tipologia de dades, es poden filtrar atributs, aplicar-hi reductors de dimensionalitat, codificar categories textuals en valors numèrics, normalitzar les dades, treure outliers...

Navegueu per la [documentació de sklearn sobre preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html) per tal de trobar les diferents opcions que proporciona sklearn.

**Preguntes:**
* Estàn les dades normalitzades? Caldria fer-ho?
* En cas que les normalitzeu, quin tipus de normalització será més adient per les vostres dades?
* Teniu gaires dades sense informació (nans)? Tingueu en compte que hi ha metodes que no els toleren durant el aprenentatge. Com afecta a la classificació si les filtrem? I si les reompliu? Com ho farieu? [Pista](https://scikit-learn.org/stable/modules/impute.html)
* Teniu dades categoriques? Quina seria la codificació amb més sentit?
* Podreu treure algun atribut extra de les categoriques (per exemple, aplicant alguna regla sobre el text)?
* Caldria aplicar PCA? Quins beneficis o inconvenients trobarieu?
* Caldria aplicar alguna tecnica de seleccio de features? Ho trobeu necessari?
* Es poden aplicar PolynomialFeatures per millorar la classificació? En quins models té sentit fer-ho?

### 3. Metric selection (1.5 punts)
En aquest apartat ens centrarem en les mètriques de classificació ([documentació](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics)). Per a fer-ho, entreneu una regressio logistica (no cal separar train-test) i a partir d'aquesta generarem una serie de funcions per analitzar els nostres resultats . Aquestes funcions ens serviran mes endevant. Caldra tambe triar la metrica que farem servir despres per triar el millor model.

**Preguntes:**
* A teoria, hem vist el resultat d'aplicar el `accuracy_score` sobre dades no balancejades. Podrieu explicar i justificar quina de les següents mètriques será la més adient pel vostre problema? `accuracy_score`, `f1_score` o `average_precision_score`?
* Abans de comencar a entrenar models, genereu una suite de funcions per poder analitzar graficament com esta anant el vostre model. Mostreu la Precisió-Recall Curve i la ROC Curve. Quina és més rellevant pel vostre dataset? Expliqueu amb les vostres paraules, la diferencia entre una i altre [Pista](https://stats.stackexchange.com/questions/338826/auprc-vs-auc-roc)
* Què mostra [classification_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)? Quina métrica us fixareu per tal de optimitzar-ne la classificació pel vostre cas?

Nota: Fixeu-vos que en aquest apartat NO ES VALOREN ELS RESULTATS. L'unic que es valora es l'eleccio de la metrica de classificacio aixi com saber quin tipus de grafiques fer per analitzar els resultats. Abans de solucionar un problema cal tenir molt clar la metrica d'error que es fara servir, i es una decisio que cal pendre previa a entrenar models

### 4. Model Selection amb validacio creuada (4 punts)

Fent servir la metrica trobada en l'apartat anterior, en aquest apartat caldra seleccionar una serie de models i, fent us de la validacio creuada, seleccionar el millor model amb els seus respectius millors hyperparametres que haurem buscat fent hyperparameter search.

La tasca d'aquesta pràctica s'enmarca dins l'aprenentatge computacional **supervisat**. A sklearn, disposem de varies tècniques [(veure documentació)](https://scikit-learn.org/stable/supervised_learning.html). A les classes de teoria, hem vist varies tècniques, com ara logistic regression, SVM amb diferents kernels, Nearest Neighbour... i tambe coneixeu altres tecniques d'altres cursos, com els arbres de decisio. Ademes, en la classe de problemes hem donat tambe els random forest i els gradient boosting. Per aquest apartat es demana seleccionar **un minim de 4 models** (per exemple, regressio logistica, random forest, KNN, SVM).

**Preguntes:**
* Quins models heu considerat? Perque els heu seleccionat?
* Fent servir validacio creuada, escolliu el millor model (agafant els hiperparamtres per defecte). Recordeu fer servir la metrica utilitzada en l'apartat anterior. Perque es important fer servir validacio creuada? Heu de fer servir algun [tipus de validacio creuada](https://scikit-learn.org/stable/modules/cross_validation.html) en especial?

* Seleccioneu una serie d'hyperparametres a provar per cadascun dels models i realitzeu una cerca d'hyperparametres. Hi ha algun model que creieu que podeu descartar de primeres? Perque?

* Mostreu els resultats en una taula on es mostri el model, els experiments realitzats i els resultats obtinguts (tant en train com en test). Podeu mostrar tambe el temps d'entrenament de cada model.

* Quin tipus de K-fold heu escollit en la seleccio de models? I en la seleccio de models amb hyperparametres? Com afecta al resultat modificar el numero de k (numero de folds) al resultat?

* Quines formes de buscar el millor parametre heu trobat? Són costoses computacionalment parlant? [documentació](https://scikit-learn.org/stable/modules/grid_search.html) Quina heu seleccionat?

* Si disposem de recursos limitats (per exemple, un PC durant 1 hora) quin dels métodes creieu que obtindrà millor resultat final?

* Existeixen altres mètodes de búsqueda més eficients ([scikit-optimize](https://scikit-optimize.github.io/stable/))?

* Opcional : Feu la prova, i amb el model i el metode de crossvalidació escollit, configureu els diferents metodes de búsqueda per a que s'executin durant el mateix temps (i.e. depenent del problema, 0,5h-1 hora). Analitzeu quin ha arribat a una millor solució. (Ajuda: estimeu el temps que trigarà a fer 1 training el vostre model, i aixi trobeu el número de intents que podeu fer en cada cas.)


### 5.Analisi Final (1.5 punt)

Un cop seleccionat el millor model amb els millors hiperparamtres, caldra fer un report final amb els resultats obtinguts.

Preguntes:
* Mostreu les curves ROC/PR (la que hageu escollit en l'apartat 2) i interpreteu els resultats.

* Analitzeu en detall les diferents metriques que trobeu adients i comenteu per sobre com podrieu fer servir aquest model en un futur. Aixo es el que es coneix com un cas d'us.

* Com creieu que es podria millorar el vostre model?